In [59]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import json

In [57]:
user_item_df = pd.read_csv('./All_Beauty/user_item.csv')
user_item_df.head()

,reviewerID,asin
0,A1V6B6TNIC10QE,0143026860
1,A2F5GHSXFQ0W6J,0143026860
2,A1572GUYS7DGSR,0143026860
3,A1PSGLFK1NSVO,0143026860
4,A6IKXKZMTKGSC,0143026860


# k-coreグラフを作る

- 少なくとも2-core
- [networkx.algorithms.core.k_core](https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.core.k_core.html#networkx.algorithms.core.k_core)


In [33]:
edges = user_item_df.values
G = nx.DiGraph()
G.add_edges_from(edges)

In [43]:
print('edge {}'.format(len(G.edges())))
print('node {}'.format(len(G.nodes())))

edge 361605
node 356624


In [50]:
k_G = nx.algorithms.core.k_core(G, 2)
print('edge {}'.format(len(k_G.edges())))
print('node {}'.format(len(k_G.nodes())))

edge 56348
node 29092


In [77]:
row = []
for edge in k_G.edges():
    row.append([edge[0], edge[1]])

df = pd.DataFrame(row, columns=['reviewerID', 'asin'])
df.to_csv('All_Beauty_2core/user_item.csv', index=False)

# Tripletを作る

In [60]:
def parse(path):
    f = open(path, 'rb')
    for l in f:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

meta_df = getDF('meta_All_Beauty.json')
#df = getDF('Luxury_Beauty.json')
#meta_df = getDF('meta_Luxury_Beauty.json')

In [78]:
# item ->(belong_to) brand
item_set = set(df['asin'])
item_brand_df = pd.concat([meta_df['asin'], meta_df['brand']], axis=1)
print(len(item_brand_df))

del_idx = []
for i in range(len(meta_df)):
    if meta_df['asin'][i] not in item_set:
        del_idx.append(i)

item_brand_df = item_brand_df.drop(del_idx)
brand = item_brand_df['brand'].values
count = 0
for b in brand:
    if type(b) != str:
        count += 1

item_brand_df.to_csv('All_Beauty_2core/item_brand.csv', index=False)

32992


In [70]:
len(set(list(df['asin'])))

4102

In [67]:
len(set(df['asin']))

4102